In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요
#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * webui/lycoris : 라이코리스(LyCORIS)를 넣어주면 읽어올수 있어요
#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요
#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Default = 'https://civitai.com/api/download/models/121931?type=Model&format=SafeTensor&size=full&fp=fp16' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_DefaultName = 'DreamShaper_XL_09Alpha.safetensors' #@param {type:"string"}
#@markdown **Ngrok**
Ngrok_Key = '' #@param {type:"string"}

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/output
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/lora
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/lycoris
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/embedding
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/wildcards

checkpoint_url = Checkpoint_Default
checkpoint_name = Checkpoint_DefaultName
checkpoint_down_path = '/content/sdd-webui/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  checkpoint_down_path = '/content/sdd-webui/models/Stable-diffusion/google'

checkpoint_file = checkpoint_down_path+'/'+checkpoint_name

%cd /content

!apt -y install -qq aria2
!pip install xformers!=0.0.18 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

!git clone -b v1.5.0 https://github.com/neuralninja22/sd-webui /content/sdd-webui
%cd /content/sdd-webui
!git reset --hard

# 구글드라이브 연결
if Google_Drive:
  !mkdir /content/sdd-webui/models/Lora
  !mkdir /content/sdd-webui/models/LyCORIS
  !mkdir /content/sdd-webui/models/hypernetworks
  !rm /content/sdd-webui/embeddings/google
  !rm /content/sdd-webui/models/Stable-diffusion/google
  !rm /content/sdd-webui/models/Lora/google
  !rm /content/sdd-webui/models/LyCORIS/google
  !rm /content/sdd-webui/models/hypernetworks/google
  !rm /content/sdd-webui/outputs
  !mv /content/sdd-webui/outputs /content/sdd-webui/outputs_backup
  !rm /content/sdd-webui/extensions/sd-dynamic-prompts/wildcards
  !mv /content/sdd-webui/extensions/sd-dynamic-prompts/wildcards /content/sdd-webui/extensions/sd-dynamic-prompts/wildcards_backup
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/embedding /content/sdd-webui/embeddings/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint /content/sdd-webui/models/Stable-diffusion/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/lora /content/sdd-webui/models/Lora/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/lycoris /content/sdd-webui/models/LyCORIS/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork /content/sdd-webui/models/hypernetworks/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/output /content/sdd-webui/outputs
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/wildcards /content/sdd-webui/extensions/sd-dynamic-prompts/wildcards
else:
  !rm /content/sdd-webui/outputs

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{checkpoint_url}" -d {checkpoint_down_path} -o {checkpoint_name}

if Ngrok_Key:
  !python launch.py --xformers --no-half-vae --api --nowebui --ngrok {Ngrok_Key}
else:
  !python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access